# Carregando Pacotes

In [ ]:
pip install pdfplumber sentence_transformers

In [ ]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: CPU


# Variáveis Globais

In [ ]:
#Tamanho da Janela de Contexto
window_size=400

#Overlap da Janela de Contexto
stride=100

#Tamanho da Chunk
chunk_size = 400

#Overlap da Chunk
overlap = 100

#Tamanho Máximo da Pergunta
tamanho_pergunta = 60

In [ ]:
modelo_tokenizador = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

# Tratamento do Documento

In [ ]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained(modelo_tokenizador)

qa_pipeline = pipeline(
    "question-answering",
    model= modelo_tokenizador,  # Modelo em Portugues
    tokenizer=modelo_tokenizador,
    device=device
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [ ]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

In [ ]:
documento = carregar_documento('Cabelo Cacheado Maior.pdf')

## Divisao em Chunks

In [ ]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

## Janela Deslizante

In [ ]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []

    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)

        # Para se chegarmos ao final do texto
        if i + window_size >= len(tokens):
            break

    return janelas

# Tokenização dos Chunks

In [ ]:
modelo_vetorizador = SentenceTransformer('all-MiniLM-L6-v2')

# Vetorizar todos os chunks
vetores = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Função Recuperação

In [ ]:
def recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto, k=3):
    # Vetoriza a pergunta
    vetor_pergunta = modelo_vetorizador.encode([pergunta], convert_to_tensor=False)

    # Vetoriza todos os chunks
    vetores_chunks = modelo_vetorizador.encode(divisao_do_texto, convert_to_tensor=False)

    # Calcula similaridade
    sim = cosine_similarity(vetor_pergunta, vetores_chunks)

    # Pega índices dos top-k chunks mais similares
    top_k_indices = np.argsort(sim[0])[::-1][:k]

    chunks_relevantes = [divisao_do_texto[i] for i in top_k_indices]

    return chunks_relevantes


# Modelo Gerador

In [ ]:
modelo_gerador = pipeline("text2text-generation", model="facebook/bart-large", device=device)

def gerar_resposta(pergunta, chunks_relevantes):
    contexto = " ".join(chunks_relevantes)
    # Limita o contexto a 800 caracteres (~aproximadamente seguro para BART)
    contexto_limitado = contexto[:800]
    prompt = f"Contexto: {contexto_limitado} \n\nPergunta: {pergunta} \n\nResposta:"
    resposta = modelo_gerador(prompt, max_length=256, truncation = True)
    return resposta[0]['generated_text']


Device set to use cpu


# Processamento

In [ ]:
documento = carregar_documento('Cabelo Cacheado Maior.pdf')

In [ ]:
#Divisao em Janela Deslizante

divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

#divisao_do_texto = dividir_em_chunks_tokenizados(documento)

In [ ]:
pergunta = "Quais são os tipos de cachos?"

chunks_relevantes = recuperar_chunks(pergunta, modelo_vetorizador, divisao_do_texto)

resposta = gerar_resposta(pergunta, chunks_relevantes)

print("📝 Resposta:", resposta)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Resposta: Contexto: ##ículo, mais apertada será a curvatura resultante, explicando por que alguns cachos são largos e soltos, enquanto outros se enrolam em espirais quase microscópicas. Os cabelos do Tipo 2, por exemplo, são considerados ondulados, transitando entre o liso e o cacheado. No entanto, é importante ressaltar que essa categorização não é rígida — muitos indivíduos possuem mais de um tipo de curl em sua cabeça, uma mistura que desafia padrões e exige cuidados personalizados. Eles geralmente apresentam um formato de " S " suave, mas podem variar desde ondas quase cerca de uma centífera para ondos mais diminuídos. A classificação das cabeças é uma das maiores formas de cacho, como o Tipo 1
